In [1]:
  
#-*- coding: utf-8 -*-

from __future__ import unicode_literals, print_function, division
from comet_ml import Experiment
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


SOS_token = 0
EOS_token = 1

# hyperparams = {
# 	"rnn_size": 128,  # assuming encoder and decoder use the same rnn_size
# 	"embedding_size": 128,
# 	"num_epochs": 20,
# 	"batch_size": 1,
# 	"learning_rate": 0.001,
# 	"max_sequence_length": 20,
# 	}

experiment = Experiment(api_key="pTKIV6STi3dM8kJmovb0xt0Ed",
                        project_name="lang2ltl", workspace="suchzheng2")

# experiment.log_parameters(hyperparams)

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )



def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    return s



def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(".\input_500_originaltext.csv").\
        read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split(',')[1:]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 20


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('lang', 'ltl', False)
#print(random.choice(pairs))


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)




class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)




def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    print(pair)
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)




teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    output_tokens = []
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            output_tokens.append(decoder_input)
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    
    if loss.item() / target_length > 10:
        predict = [output_lang.index2word[id] for id in output_tokens]
        target = [output_lang.index2word[id] for id in target_tensor]
        print("="," ",join(predict))
        print("<"," ",join(target))


    return loss.item() / target_length
    #return loss


import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))




def trainIters(encoder, decoder,  training_pairs, print_every=1000, plot_every=100, learning_rate=0.01, val_every = 100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    criterion = nn.NLLLoss()
    
    n_iters = len(training_pairs)
    curr_loss = []
    spikes = []
    with experiment.train():
        
        for iter in range(1, n_iters + 1):
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)

            experiment.log_metric("train_loss",loss)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

#             if iter % val_every == 0:
#                 evaluate_model(encoder1, attn_decoder1,validation_pairs)

            # curr_loss += loss 

        # curr_loss.backward()
        
        # encoder_optimizer.step()
        # decoder_optimizer.step()
    #showPlot(plot_losses)


######################################################################
# Plotting results
# ----------------
#
# Plotting is done with matplotlib, using the array of loss values
# ``plot_losses`` saved while training.
#

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


######################################################################
# Evaluation
# ==========
#
# Evaluation is mostly the same as training, but there are no targets so
# we simply feed the decoder's predictions back to itself for each step.
# Every time it predicts a word we add it to the output string, and if it
# predicts the EOS token we stop there. We also store the decoder's
# attention outputs for display later.
#

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
    
        decoder_attentions = torch.zeros(max_length, max_length)
        decoder_outputs = torch.zeros(max_length,output_lang.n_words)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            decoder_outputs[di] = decoder_output
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1],decoder_outputs


######################################################################
# We can evaluate random sentences from the training set and print out the
# input, target, and output to make some subjective quality judgements:
#

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

def evaluate_model(encoder, decoder,evaluate_pairs):
    criterion = nn.NLLLoss()
    cur_loss = 0
    num_target = 0
    with experiment.validate(), torch.no_grad():
        for pair in evaluate_pairs:
            print('>', pair[0])
            print('=', pair[1])
            output_words, attentions, decoder_outputs = evaluate(encoder, decoder, pair[0])
            output_sentence = ' '.join(output_words)
            print('<', output_sentence)
            print('')

            target_tensor = tensorFromSentence(output_lang, pair[1])
            target_tensor = target_tensor.to(device)
            sentence_loss = 0
            for di in range(len(target_tensor)):
                loss = criterion(decoder_outputs[di].unsqueeze(0).to(device),target_tensor[di].to(device))
                #experiment.log_metric("loss ", loss)
                sentence_loss += loss
                cur_loss += loss
                
            
            #perplexity = torch.exp(sentence_loss/len(target_tensor))
            #experiment.log_metric("validation perplexity ", perplexity)
            num_target += len(target_tensor)
        
        #print("validation curr loss: ",cur_loss)
        print("validation loss: ",cur_loss/num_target)
        experiment.log_metric("loss: ",cur_loss/num_target)
        return cur_loss/num_target
        #print("validation perplexity: ",perplexity)




COMET INFO: Experiment is live on comet.ml https://www.comet.ml/suchzheng2/lang2ltl/c99d77bf2d434290be1123565a19576b



Reading lines...
Read 556 sentence pairs
Counting words...
Counted words:
lang 45
ltl 35


In [2]:
####################################################################
# train the model
####################################################################
hidden_size = 256
num_epoch = 5
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
train_split = 0.8
valid_split = 0.1

random.shuffle(pairs)
num_train_pairs = int(len(pairs)*0.8)
num_valid_pairs = int(len(pairs)*0.1)
num_test_pairs = int(len(pairs) - num_train_pairs - num_valid_pairs)

training_pairs = pairs[:num_train_pairs]
validation_pairs = pairs[num_train_pairs:num_train_pairs+num_valid_pairs]
testing_pairs = pairs[num_train_pairs+num_valid_pairs:]

training_pairs = [tensorsFromPair(pair)
                      for pair in pairs[:num_train_pairs]]
for i in range(num_epoch):
    print("=============== Epoch: %d =================="%i)
    random.shuffle(training_pairs)
    trainIters(encoder1, attn_decoder1, training_pairs, print_every=100,val_every = 10)
    evaluate_model(encoder1, attn_decoder1,validation_pairs)



['go to the watson center and then go to the science library', 'f ( watson center & f ( science library ) )']
['find the science library and then find the starbucks', 'f ( science library & f ( starbucks ) )']
["stay on the brook st first find the marston hall and then find the citizen's bank", "g ( brook st ) & f ( marston hall & f ( citizen's bank ) )"]
["do not enter thayer st first find the marston hall and then find the citizen's bank", "g ( ! thayer st ) & f ( marston hall & f ( citizen's bank ) )"]
['find the barus building and then find the science library', 'f ( barus building & f ( science library ) )']
['find the starbucks and then find the watson center', 'f ( starbucks & f ( watson center ) )']
['find the marston hall and then find the nelson fitness center', 'f ( marston hall & f ( nelson fitness center ) )']
['go to the bookstore and then go to the barus building', 'f ( bookstore & f ( barus building ) )']
['find the cvs and then find the bookstore', 'f ( cvs & f ( books

['do not enter brook st first find the cvs and then find the chipotle', 'g ( ! brook st ) & f ( cvs & f ( chipotle ) )']
['find the science library and then find the cvs', 'f ( science library & f ( cvs ) )']
['go to the nelson fitness center', 'f ( nelson fitness center )']
['stay on the thayer st first find the bookstore and then find the starbucks', 'g ( thayer st ) & f ( bookstore & f ( starbucks ) )']
['find the chipotle and then find the science library', 'f ( chipotle & f ( science library ) )']
['do not enter brook st first find the bookstore and then find the chipotle', 'g ( ! brook st ) & f ( bookstore & f ( chipotle ) )']
["go to the cvs and then go to the citizen's bank", "f ( cvs & f ( citizen's bank ) )"]
['stay on the angell st first find the chipotle and then find the bookstore', 'g ( angell st ) & f ( chipotle & f ( bookstore ) )']
['do not enter brook st first find the science library and then find the chipotle', 'g ( ! brook st ) & f ( science library & f ( chipotle 

0m 3s (- 0m 11s) (100 22%) 2.1137
0m 6s (- 0m 7s) (200 45%) 1.6401
0m 9s (- 0m 4s) (300 67%) 1.6489
0m 13s (- 0m 1s) (400 90%) 1.4375
> find the fedex office and then find the marston hall
= f ( fedex office & f ( marston hall ) )
< f ( kabob and curry & f ( ( ) <EOS>

> do not enter brook st first find the bookstore and then find the starbucks
= g ( ! brook st ) & f ( bookstore & f ( starbucks ) )
< g ( ! brook st ) & f ( ( kabob and curry ) <EOS>

> do not enter thayer st first find the bookstore and then find the chipotle
= g ( ! thayer st ) & f ( bookstore & f ( chipotle ) )
< g ( ! brook st ) & f ( ( kabob and curry ) <EOS>

> find the starbucks and then find the barus building
= f ( starbucks & f ( barus building ) )
< f ( kabob and curry & f ( ( ) <EOS>

> do not enter thayer st first find the marston hall and then find the watson center
= g ( ! thayer st ) & f ( marston hall & f ( watson center ) )
< g ( ! brook st ) & f ( kabob and curry & f ( ( ) <EOS>

> go to the nelson fit

0m 3s (- 0m 11s) (100 22%) 1.2468
0m 6s (- 0m 8s) (200 45%) 0.9993
0m 9s (- 0m 4s) (300 67%) 1.1736
0m 13s (- 0m 1s) (400 90%) 1.0414
> find the fedex office and then find the marston hall
= f ( fedex office & f ( marston hall ) )
< f ( nelson fitness center & f ( nelson fitness center ) ) <EOS>

> do not enter brook st first find the bookstore and then find the starbucks
= g ( ! brook st ) & f ( bookstore & f ( starbucks ) )
< g ( ! brook st ) & f ( kabob and curry ( ) ) <EOS>

> do not enter thayer st first find the bookstore and then find the chipotle
= g ( ! thayer st ) & f ( bookstore & f ( chipotle ) )
< g ( ! thayer st ) & f ( kabob and curry ( ) ) <EOS>

> find the starbucks and then find the barus building
= f ( starbucks & f ( barus building ) )
< f ( starbucks & f ( nelson fitness center ) ) <EOS>

> do not enter thayer st first find the marston hall and then find the watson center
= g ( ! thayer st ) & f ( marston hall & f ( watson center ) )
< g ( ! brook st ) & f ( marsto

0m 3s (- 0m 10s) (100 22%) 0.8541
0m 6s (- 0m 8s) (200 45%) 0.6972
0m 9s (- 0m 4s) (300 67%) 0.7150
0m 13s (- 0m 1s) (400 90%) 0.4930
> find the fedex office and then find the marston hall
= f ( fedex office & f ( marston hall ) )
< f ( fedex office & f ( marston hall ) ) <EOS>

> do not enter brook st first find the bookstore and then find the starbucks
= g ( ! brook st ) & f ( bookstore & f ( starbucks ) )
< g ( ! brook st ) & f ( bookstore & f ( chipotle ) ) <EOS>

> do not enter thayer st first find the bookstore and then find the chipotle
= g ( ! thayer st ) & f ( bookstore & f ( chipotle ) )
< g ( ! thayer st ) & f ( bookstore & f ( chipotle ) ) <EOS>

> find the starbucks and then find the barus building
= f ( starbucks & f ( barus building ) )
< f ( starbucks & f ( barus building ) ) <EOS>

> do not enter thayer st first find the marston hall and then find the watson center
= g ( ! thayer st ) & f ( marston hall & f ( watson center ) )
< g ( ! thayer st ) & f ( marston hall & f

0m 3s (- 0m 11s) (100 22%) 0.3269
0m 6s (- 0m 8s) (200 45%) 0.2096
0m 9s (- 0m 4s) (300 67%) 0.2063
0m 13s (- 0m 1s) (400 90%) 0.1818
> find the fedex office and then find the marston hall
= f ( fedex office & f ( marston hall ) )
< f ( fedex office & f ( marston hall ) ) <EOS>

> do not enter brook st first find the bookstore and then find the starbucks
= g ( ! brook st ) & f ( bookstore & f ( starbucks ) )
< g ( ! brook st ) & f ( bookstore & f ( starbucks ) ) <EOS>

> do not enter thayer st first find the bookstore and then find the chipotle
= g ( ! thayer st ) & f ( bookstore & f ( chipotle ) )
< g ( ! thayer st ) & f ( bookstore & f ( starbucks ) ) <EOS>

> find the starbucks and then find the barus building
= f ( starbucks & f ( barus building ) )
< f ( starbucks & f ( barus building ) ) <EOS>

> do not enter thayer st first find the marston hall and then find the watson center
= g ( ! thayer st ) & f ( marston hall & f ( watson center ) )
< g ( ! thayer st ) & f ( marston hall &

0m 3s (- 0m 11s) (100 22%) 0.1422
0m 6s (- 0m 8s) (200 45%) 0.1662
0m 10s (- 0m 4s) (300 67%) 0.0490
0m 13s (- 0m 1s) (400 90%) 0.0510
> find the fedex office and then find the marston hall
= f ( fedex office & f ( marston hall ) )
< f ( fedex office & f ( marston hall ) ) <EOS>

> do not enter brook st first find the bookstore and then find the starbucks
= g ( ! brook st ) & f ( bookstore & f ( starbucks ) )
< g ( ! brook st ) & f ( bookstore & f ( starbucks ) ) <EOS>

> do not enter thayer st first find the bookstore and then find the chipotle
= g ( ! thayer st ) & f ( bookstore & f ( chipotle ) )
< g ( ! thayer st ) & f ( bookstore & f ( starbucks ) ) <EOS>

> find the starbucks and then find the barus building
= f ( starbucks & f ( barus building ) )
< f ( starbucks & f ( barus building ) ) <EOS>

> do not enter thayer st first find the marston hall and then find the watson center
= g ( ! thayer st ) & f ( marston hall & f ( watson center ) )
< g ( ! thayer st ) & f ( marston hall 

In [3]:
evaluate_model(encoder1, attn_decoder1,validation_pairs)

> find the fedex office and then find the marston hall
= f ( fedex office & f ( marston hall ) )
< f ( fedex office & f ( marston hall ) ) <EOS>

> do not enter brook st first find the bookstore and then find the starbucks
= g ( ! brook st ) & f ( bookstore & f ( starbucks ) )
< g ( ! brook st ) & f ( bookstore & f ( starbucks ) ) <EOS>

> do not enter thayer st first find the bookstore and then find the chipotle
= g ( ! thayer st ) & f ( bookstore & f ( chipotle ) )
< g ( ! thayer st ) & f ( bookstore & f ( chipotle ) ) <EOS>

> find the starbucks and then find the barus building
= f ( starbucks & f ( barus building ) )
< f ( starbucks & f ( barus building ) ) <EOS>

> do not enter thayer st first find the marston hall and then find the watson center
= g ( ! thayer st ) & f ( marston hall & f ( watson center ) )
< g ( ! thayer st ) & f ( marston hall & f ( watson center ) ) <EOS>

> go to the nelson fitness center and then go to the fedex office
= f ( nelson fitness center & f ( fedex

> stay on the brook st first find the watson center and then find the marston hall
= g ( brook st ) & f ( watson center & f ( marston hall ) )
< g ( brook st ) & f ( watson center & f ( marston hall ) ) <EOS>

> do not enter brook st and find the kabob and curry
= g ( ! brook st ) & f ( kabob and curry )
< g ( ! brook st ) & f ( kabob and curry ) <EOS>

> find the watson center and then find the fedex office
= f ( watson center & f ( fedex office ) )
< f ( watson center & f ( fedex office ) ) <EOS>

validation loss:  tensor(0.0296, device='cuda:0')


tensor(0.0296, device='cuda:0')

https://www.comet.ml/suchzheng2/lang2ltl/962a74a529d24149b3ad28d584b6d938?experiment-tab=chart&showOutliers=true&smoothing=0&transformY=smoothing&xAxis=step

In [4]:
def evaluateExample(input_sentence):
    output_words = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    #print(output_words)
    print('output =', ' '.join(output_words[0]))
    #showAttention(input_sentence, output_words, attentions)


In [12]:
evaluateExample("stay on the thayer st and find the bookstore")

input = stay on the thayer st and find the bookstore
output = g ( thayer st ) & f ( bookstore ) <EOS>


In [13]:
evaluateExample("find the bookstore")
evaluateExample("find the starbucks and then find the science library")

input = find the bookstore
output = f ( bookstore ) <EOS>
input = find the starbucks and then find the science library
output = f ( starbucks & f ( science library ) ) <EOS>


In [15]:
evaluateExample("go to the starbucks")

input = go to the starbucks
output = f ( starbucks & f ( starbucks ) ) <EOS>


In [16]:
evaluateExample("find the bookstore")

input = find the bookstore
output = f ( bookstore ) <EOS>


In [17]:
evaluateExample("find bookstore")

input = find bookstore
output = f ( bookstore ) <EOS>


In [7]:
evaluateExample("stay on the brook st first find the citizen's bank and then find the barus building")

input = stay on the brook st first find the citizen's bank and then find the barus building
output = g ( brook st ) & f ( citizen's bank & f ( barus building ) ) <EOS>


In [8]:
evaluateExample("stay on the thayer st and find the bookstore")

input = stay on the thayer st and find the bookstore
output = g ( thayer st ) & f ( bookstore ) <EOS>


In [9]:
evaluateExample("stay on the starbucks and find the bookstore")

input = stay on the starbucks and find the bookstore
output = f ( starbucks & f ( starbucks ) ) <EOS>


In [10]:
evaluateExample("stay on the citizen's bank and find the bookstore")

input = stay on the citizen's bank and find the bookstore
output = f ( citizen's bank & f ( bookstore ) ) <EOS>


In [11]:
evaluateExample("stay on the thayer st and find the brook st")

input = stay on the thayer st and find the brook st
output = g ( thayer st ) & f ( citizen's bank ) <EOS>
